<a href="https://colab.research.google.com/github/Vizzuality/copernicus-climate-data/blob/master/cds_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Linux dependencies

## Python packages

In [0]:
!pip install cftime netcdf4 zarr xarray xclim cdsapi cdstoolbox-remote

     |████████████████████████████████| 3.3MB 2.8MB/s 
     |████████████████████████████████| 3.8MB 33.2MB/s 
  Created wheel for zarr: filename=zarr-2.4.0-cp36-none-any.whl size=127066 sha256=04770c8d838f2a57584817dc901889faa063fdda791a57cd52d7fabd72fe4089
  Stored in directory: /root/.cache/pip/wheels/e1/5b/25/24c685604b91139aba00a5b6299b53e7a0661f737f27782559
  Created wheel for asciitree: filename=asciitree-0.3.3-cp36-none-any.whl size=5037 sha256=aba747db9907186ba367694654b96eb6d449580d99cfd81b08faef4fa3044eb7
  Stored in directory: /root/.cache/pip/wheels/1d/d9/58/9808b306744df0208fccc640d3d9952a5bc7468502d42897d5
  Created wheel for numcodecs: filename=numcodecs-0.6.4-cp36-cp36m-linux_x86_64.whl size=3885735 sha256=63a8f24bcc660bdc6db6bb87c2936eabb8e04ad7e4ed68528fe96c6ab650825a
  Stored in directory: /root/.cache/pip/wheels/ca/07/ed/fea2e120cbb91d90b577c5ac56b4b082024f56fcd88e9afa55
Successfully built zarr asciitree numcodecs


## Authorisation

In [0]:
# For auth WITHOUT service account
#from google.colab import auth
#auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
#project_id = "skydipper-196010"
#!gcloud config set project {project_id}

In [0]:
# Copy GC credentials to home (place in your GDrive, and connect Drive)
!cp "/content/drive/My Drive/skydipper-196010-f842645fd0f3.json" "/root/.skydipper-196010-f842645fd0f3.json"

In [0]:
# Auth WITH service account
!gcloud auth activate-service-account \
  edward-morris@skydipper-196010.iam.gserviceaccount.com \
          --key-file=/root/.skydipper-196010-f842645fd0f3.json --project="skydipper-196010"


Activated service account credentials for: [edward-morris@skydipper-196010.iam.gserviceaccount.com]


To take a quick anonymous survey, run:
  $ gcloud survey



In [0]:
# Test GC auth
!gsutil ls "gs://copernicus-climate"

gs://copernicus-climate/RH_ERA5.zip
gs://copernicus-climate/variables_futuro_CMIP5_AEMET.zip
gs://copernicus-climate/variables_futuro_CMIP5_ERA5land_1.zip
gs://copernicus-climate/variables_futuro_CMIP5_ERA5land_2.zip
gs://copernicus-climate/variables_futuro_CMIP5_ERA5land_3.zip
gs://copernicus-climate/variables_futuro_CMIP5_ERA5land_4.zip
gs://copernicus-climate/variables_futuro_seasonal_ERA5land_1.zip
gs://copernicus-climate/variables_futuro_seasonal_ERA5land_2.zip
gs://copernicus-climate/variables_futuro_seasonal_ERA5land_3.zip
gs://copernicus-climate/ws_ERA5.zip
gs://copernicus-climate/dataset/
gs://copernicus-climate/future-daily/
gs://copernicus-climate/spain.zarr/


In [0]:
# Copy CDS API credentials to home (place in your GDrive, and connect Drive)
!cp "/content/drive/My Drive/cdsapirc" "/root/.cdsapirc"


In [0]:
# Clean up
!rm -r sample_data

# Utils

## copy_gcs

In [0]:
import os
import subprocess

def copy_gcs(source_list, dest_list, opts=""):
  """
  Use gsutil to copy each corresponding item in source_list
  to dest_list

  """
  for s, d  in zip(source_list, dest_list):
    cmd = f"gsutil -m cp -r {opts} {s} {d}"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
  print("Finished copy")

## list_paths

In [0]:
import glob
import subprocess

def list_paths(uri_prefix, dir_path, file_pattern="*", gsutil=True, return_dir_path=True):
        ''' Creates a list of full paths 
    
        Uses glob regex rules allowing flexible patterns
    
        Parameters
        ----------
        uri_prefix : str
            The (GCS) uri prefix.
        dir_path : str
            Directory path, can use regex.
        file_pattern : str
            File pattern for glob searching.
        gsutil : bool
            Use gsutil, default is True.
        return_dir_path : bool
            Return directory path relative to uri_prefix, default is True.        
    
        Returns
        -------
        List of path strings.
        
        Examples
        --------
        # Requires authentication
        #list_paths("gs://skydipper-water-quality", "cloud-masks/*", "*.tif", True, False)
        '''
        p = f"{uri_prefix}/{dir_path}/{file_pattern}"
        print(f"Searching {p}")
        if not gsutil:
          out = glob.glob(p)
        if gsutil:
          cmd = f"gsutil ls {p}"
          out = subprocess.check_output(cmd, shell=True).decode('utf8').split('\n')
          out.pop(-1)
        if return_dir_path:
          out = [f.split(uri_prefix)[1] for f in out]  
        print(f"Found {len(out)} path(s)")
        return out

#path_list = list_paths("gs://skydipper-water-quality", "cloud-masks/*", "*.tif", True, True)
#print(path_list[0])



## extract_string

In [0]:
import os
def extract_string(file_path, split, index):
  """
  Get string by splitting path
  
  @arg file_path The file path string to split
  @arg split Caracter to split path on
  @index Index integer to retain

  @return A string
  """ 
  return os.path.splitext(file_path)[0].split(split)[index]

## set_acl_to_public

In [0]:
import subprocess

# Set to asset permissions to public for https read
def set_acl_to_public(gs_path):
  """ 
  Set all Google Storage assets to puplic read access.

  Requires GS authentication

  Parameters
  ----------
  gs_path str
    The google storage path, note the "-r" option is used, setting the acl of all assets below this path
  """
  cmd = f"gsutil -m acl -r ch -u AllUsers:R {gs_path}"
  print(cmd)
  r = subprocess.call(cmd, shell=True)
  if r is 0:
    print("Set acl(s) sucsessful")
  else:
    print("Set acl(s) failed")  

#set_acl_to_public("gs://skydipper-water-quality/cloud-masks")

## extract_zip

In [0]:
# Extract only what we want
from zipfile import ZipFile

def extract_zip(zip_path, dest_path, patterns):
  with ZipFile(zip_path, 'r') as zipObj:
      # Get a list of all archived file names from the zip
      fl = zipObj.namelist()
      #print(fl)
      for pattern in patterns:
          for f in fl:
              if pattern in f:
                print(f)
                zipObj.extract(f, dest_path)
  print("Finished extraction")

## copy_zip_extract_rm

In [0]:
import os
def copy_zip_extract_rm(src_path, dest_path, extract_path, patterns):
  print(f"Copying {src_path} to {dest_path}")
  copy_gcs([src_path], [dest_path], opts="-n")
  zp = os.path.join(dest_path, os.path.basename(src_path))
  print(f"Extracting {zp} with pattern {patterns} to {extract_path}")
  extract_zip(zp, extract_path, patterns)
  print(f"Removing {zp}")
  os.remove(zp)
  print("Finished copy, extract, and remove")


# Data package structure

+ CMIP5 future predictions, datetimes are not correct, they are simply a placeholder for the daily data.
+ There are 4 scenarios, we only require "rcp45" and "rcp85".
+ There are various models per scenario, eventually we require the mean and sd of the models
+ Data are clipped to a single region (Spain)
+ Seperate files for tasmax and tasmin


```
*.zip
  [variable]_[model]_[scenario]_[country]_[period].nc  
```


# Data packages

1. Transform and download using the CDS toolbox or, if unavailable in the Toolbox, use the CDS API.
1. Convert to min-max temperature at daily temporal resolution.
1. Interploate to same grid size


+ Bounding box Spain (mainland): [44.0, -10.0, 35.0, 5.0] #NWSE


+ ERA5-Land hourly data from 1981 to present
    + 2m temperature
    + grid [0.1, 0.1]
    + years 1981--2020, all months, days, and hours
    + convert to daily min/max using CDS toolbox

+ CMIP5 daily data on single levels
    + variable grid size
    + experiments 'rcp_4_5', 'rcp_8_5'
    + 'minimum_2m_temperature_in_the_last_24_hours', 'maximum_2m_temperature_in_the_last_24_hours'
    + models 'bnu_esm', 'access1_0', 'mpi_esm_lr', 'mpi_esm_mr', 'noresm1_m'
    + realizations 'r1i1p1'
    + extract bbox using cds toolbox

+ Thermal-comfort-indices-derived-from-ERA5-reanalysis
    + "Mean radiant temperature", 'Universal thermal climate index'
    + years 1979--2020, all months, days, and hours
    + grid [0.25, 0.25]
    + extract bbox using cds toolbox 

In [0]:
# Define paths
ds_dir = "dataset"
gs_prefix ="gs://copernicus-climate"           

## CMIP5 daily data on single levels

### Get data from CDS toolbox

In [0]:
%%writefile download.py
import cdstoolbox as ct

# Out directory
ds_dir = "dataset"

# Define region of interest
bbox = [35, -10, 44, 5]
grid = [0.1, 0.1]

# Define datasets to download
exp_list = ['rcp_4_5', 'rcp_8_5']
var_list = ['minimum_2m_temperature_in_the_last_24_hours', 'maximum_2m_temperature_in_the_last_24_hours']
model_list = ['bnu_esm', 'access1_0', 'mpi_esm_lr', 'mpi_esm_mr', 'noresm1_m']
ensemble_list = ['r1i1p1']
period_list = [
               ['20060101-21001231'],
               ['20060101-20301231', '20310101-20551231', '20560101-20801231','20810101-21001231'],
               ['20060101-20091231', '20100101-20191231', '20200101-20291231',
                '20300101-20391231', '20400101-20491231', '20500101-20591231',
                '20600101-20691231', '20700101-20791231', '20800101-20891231',
                '20900101-21001231'],
               ['20060101-20091231', '20100101-20191231', '20200101-20291231',
                '20300101-20391231', '20400101-20491231', '20500101-20591231',
                '20600101-20691231', '20700101-20791231', '20800101-20891231',
                '20900101-21001231'],
               ['20060101-20551231', '20560101-21001231']
               ]


# Function to create properties dict for download
def create_cmip5_dict(experiment, model, ensemble, variable, period):
  return {
        'ensemble_member': ensemble,
        'experiment': experiment,
        'variable': [variable],
        'model': model,
        'period': period
    }

# Make list of properties dicts, and out names
pdict_list = []
nm_list = []
for experiment in exp_list:
  for variable in var_list:
    for model, period in zip(model_list, period_list):
      for ensemble in ensemble_list:
        pdict_list.append(create_cmip5_dict(experiment, model, ensemble, variable, period))
        
        # make file names
        if len(period) >1:
          ti = f"{period[0].split('-')[0]}-{period[-1].split('-')[1]}"
        else:
          ti = period[0] 
        e = experiment.replace("_", "-")
        m = model.replace("_", "-")
        v = variable.replace("_", "-")
        en = ensemble.replace("_", "-")
        nm_list.append(f"{ds_dir}/bbox-{bbox[0]}-{bbox[1]}-{bbox[2]}-{bbox[3]}_{e}_{m}_{en}_{v}_{ti}.nc")
print(pdict_list[0])

@ct.application(title='Get CMIP5')
@ct.output.download()
def application():
    
    #ref_grid = ct.catalogue.retrieve(
    #    "reanalysis-era5-land",\
    #     {"variable" : "2m_temperature","time" : '12:00',"day" : '15',"month" : '08', "year" : '2000', 'area': bbox, 'grid': grid})

    def get_data(pdict, nm):
      ds = ct.catalogue.retrieve('projections-cmip5-daily-single-levels', pdict)
      dss = ct.cube.box_select(ds,lat=[bbox[0],bbox[2]], lon=[bbox[1],bbox[3]])
      dsss = ct.cdm.update_attributes(dss, pdict)
      #dssss = ct.geo.regrid(dsss, reference_grid_data = ref_grid)
      return dsss
    
    return [get_data(pdict, nm) for pdict, nm in zip(pdict_list, nm_list)]



Writing download.py


In [0]:
%%time
!python download.py

2020-03-03 15:41:32,553 INFO Welcome to the CDS
2020-03-03 15:41:32,554 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/tasks/services/tool/toolbox/orchestrator/run_workflow/clientid-b90650e3323a4306ab4aabc537526f7c
2020-03-03 15:41:33,206 INFO Request is completed
2020-03-03 15:41:33,206 INFO Downloading http://136.156.132.235/cache-compute-0006/cache/data9/cdm.update_attributes-1583105491.5020332-14551-28-075dce16-6f10-4659-8f31-31540de29181updated_attrs.nc to cdm.update_attributes-1583105491.5020332-14551-28-075dce16-6f10-4659-8f31-31540de29181updated_attrs.nc (2.3M)
2020-03-03 15:41:34,054 INFO Download rate 2.7M/s
2020-03-03 15:41:34,054 INFO Downloading http://136.156.133.37/cache-compute-0011/cache/data1/cdm.update_attributes-1583224018.170093-32545-21-487bb3a9-27b3-4756-996a-8375a2b8b103updated_attrs.nc to cdm.update_attributes-1583224018.170093-32545-21-487bb3a9-27b3-4756-996a-8375a2b8b103updated_attrs.nc (8.8M)
2020-03-03 15:41:35,452 INFO Download rate 6.3M/

### Rename netcdfs

In [0]:
import xarray as xr
import glob
import os
import pprint
xr.set_options(display_style='html')

fl = glob.glob("*.nc")
print(fl)
for f in fl:
  ds = xr.open_dataset(f)
  ds = xr.decode_cf(ds)
  attrs = ds.attrs
  #pprint.pprint(attrs)
  if attrs['comment'].split(" ")[0] == 'maximum':
    v = 'tasmax'
  if attrs['comment'].split(" ")[0] == 'minimum':
    v = 'tasmin'
  dvattr = ds[v].attrs
  #pprint.pprint(dvattr)
  try:
    m = dvattr['model_id']
    e = dvattr['experiment_id']
    r = dvattr['parent_experiment_rip']
    nm = f"daily_projections-cmip5-daily-single-levels_{v}_{m}_{e}_{r}.nc"
  except:
    txt = dvattr['associated_files'].split('areacella:')[1]
    m = txt.split("_")[2]
    e = txt.split("_")[3]
    r = txt.split("_")[4]
    nm = f"daily_projections-cmip5-daily-single-levels_{v}_{m}_{e}_{r}"

  print(nm)
  ds.to_netcdf(nm)

['cdm.update_attributes-1583224022.0802572-17537-34-15ddf416-f6ad-4e4b-8aef-d69016eb0dcdupdated_attrs.nc', 'cdm.update_attributes-1583224309.65743-2661-30-180d1080-49aa-4c5b-b630-0c54353a7b72updated_attrs.nc', 'cdm.update_attributes-1583105491.5020332-14551-28-075dce16-6f10-4659-8f31-31540de29181updated_attrs.nc', 'cdm.update_attributes-1583224117.041823-11899-33-a1d207a6-cc60-4f06-9297-7ced56aee330updated_attrs.nc', 'cdm.update_attributes-1583224258.8644996-13789-38-ad5e2c57-7201-4277-ad46-482dcfe784fcupdated_attrs.nc', 'cdm.update_attributes-1583224316.5325725-26301-35-a450a1fc-0145-47a7-9f9a-b3714ce7b7b3updated_attrs.nc', 'cdm.update_attributes-1583224289.3021362-14066-38-2546732a-8551-448d-8fef-3bc414c497d5updated_attrs.nc', 'cdm.update_attributes-1583224124.7870688-16505-30-81064b06-ce98-4c7f-b8a6-da4b85f10b4dupdated_attrs.nc', 'cdm.update_attributes-1583224018.170093-32545-21-487bb3a9-27b3-4756-996a-8375a2b8b103updated_attrs.nc', 'cdm.update_attributes-1583224255.5815945-15857-38

### Merge, fix issues, and rename netcdfs

In [0]:
# Make dataset dir
!rm -r {ds_dir}
!mkdir {ds_dir}

rm: cannot remove 'dataset': No such file or directory


In [0]:
import xarray as xr
import numpy as np
import glob
import os

def concat_dicts(*dicts):
    keys = set().union(*dicts)
    #print(keys)
    return { k: ", ".join(list(set(dic.get(k) for dic in dicts)))  for k in keys}

def merge_vars(m, s):
  with xr.open_dataset(glob.glob(f"daily_projections-cmip5-daily-single-levels_tasmin_{m}_{s}_*.nc")[0])as ds1:
    ds1.coords['experiment'] = s
    ds1.coords['model'] = m
    ds1.coords['realization'] = np.array([1])
  
  with xr.open_dataset(glob.glob(f"daily_projections-cmip5-daily-single-levels_tasmax_{m}_{s}_*.nc")[0]) as ds2:
    ds2.coords['experiment'] = s
    ds2.coords['model'] = m
    ds2.coords['realization'] = np.array([1])

  out = xr.merge([ds1, ds2])
  out.set_coords(['lat', 'lon', 'time', 'experiment', 'model', 'realization'])
  out.attrs = concat_dicts(ds1.attrs, ds2.attrs)
  out.attrs['source'] = ", ".join([out.attrs['source'], 'CDS']) 
  return out

def match_grid(ds_target, ds_source):
  return ds_target.interp(lat=ds_source['lat'], lon=ds_source['lon'])

def fix_time(ds):
  a ={'standard_name': 'time',
  'long_name':         'time',
  'bounds':            'time_bnds',
  'axis':              'T',
  'stored_direction':  'increasing',
  'type':              'double'}
  ds = ds.sel(time=slice('2006-01-01', '2099-12-31'))
  t = ds.coords['time'].attrs = a
  t = ds.coords['time']
  ds.coords['time'] = xr.cftime_range(start='2006', periods=len(t.values), freq='D', calendar='noleap')
  return xr.decode_cf(ds)  


fl = glob.glob('daily_projections-cmip5-daily-single-levels_*.nc')
print(fl)
models = list(set([f.split("_")[3] for f in fl]))
variables = list(set([f.split("_")[2] for f in fl]))
scenarios = list(set([f.split("_")[4] for f in fl]))
print(models)
print(variables)
print(scenarios)

ds_source = xr.open_dataset('daily_projections-cmip5-daily-single-levels_tasmin_ACCESS1-0_rcp45_r0i0p0.nc')

for s in scenarios:
  print(s)
  os.mkdir(os.path.join(ds_dir, s))
  for m in models:
    print(m)
    os.mkdir(os.path.join(ds_dir, s, m))
    out = merge_vars(m,s)
    out = match_grid(out, ds_source)
    out = fix_time(out)
    print(out.time) 
    out.to_netcdf(os.path.join(ds_dir, s, m, f"daily_projections-cmip5-daily-single-levels_tasmin-tasmax_{m}_{s}_r1i1p1.nc"))

ds_source.close()    

['daily_projections-cmip5-daily-single-levels_tasmax_ACCESS1-0_rcp45_r0i0p0.nc', 'daily_projections-cmip5-daily-single-levels_tasmin_MPI-ESM-MR_rcp85_r0i0p0.nc', 'daily_projections-cmip5-daily-single-levels_tasmin_BNU-ESM_rcp45_r1i1p1.nc', 'daily_projections-cmip5-daily-single-levels_tasmin_MPI-ESM-LR_rcp85_r0i0p0.nc', 'daily_projections-cmip5-daily-single-levels_tasmax_MPI-ESM-LR_rcp85_r0i0p0.nc', 'daily_projections-cmip5-daily-single-levels_tasmin_NorESM1-M_rcp85_r0i0p0.nc', 'daily_projections-cmip5-daily-single-levels_tasmax_MPI-ESM-LR_rcp45_r0i0p0.nc', 'daily_projections-cmip5-daily-single-levels_tasmax_BNU-ESM_rcp45_r1i1p1.nc', 'daily_projections-cmip5-daily-single-levels_tasmin_ACCESS1-0_rcp85_r0i0p0.nc', 'daily_projections-cmip5-daily-single-levels_tasmax_BNU-ESM_rcp85_r1i1p1.nc', 'daily_projections-cmip5-daily-single-levels_tasmax_ACCESS1-0_rcp85_r0i0p0.nc', 'daily_projections-cmip5-daily-single-levels_tasmin_BNU-ESM_rcp85_r1i1p1.nc', 'daily_projections-cmip5-daily-single-level

### Open consolidated netcdf dataset

In [0]:
import os
import glob

fl = [[glob.glob(f"{ds_dir}/{s}/{m}/*.nc") for m in models] for s in scenarios]
print(fl)
ds = xr.open_mfdataset(fl, combine='nested', concat_dim=['experiment', 'model', 'realization'])\
.sel(time=slice('2006-01-01', '2099-12-31'))

[[['dataset/rcp45/NorESM1-M/daily_projections-cmip5-daily-single-levels_tasmin-tasmax_NorESM1-M_rcp45_r1i1p1.nc'], ['dataset/rcp45/BNU-ESM/daily_projections-cmip5-daily-single-levels_tasmin-tasmax_BNU-ESM_rcp45_r1i1p1.nc'], ['dataset/rcp45/MPI-ESM-MR/daily_projections-cmip5-daily-single-levels_tasmin-tasmax_MPI-ESM-MR_rcp45_r1i1p1.nc'], ['dataset/rcp45/MPI-ESM-LR/daily_projections-cmip5-daily-single-levels_tasmin-tasmax_MPI-ESM-LR_rcp45_r1i1p1.nc'], ['dataset/rcp45/ACCESS1-0/daily_projections-cmip5-daily-single-levels_tasmin-tasmax_ACCESS1-0_rcp45_r1i1p1.nc']], [['dataset/rcp85/NorESM1-M/daily_projections-cmip5-daily-single-levels_tasmin-tasmax_NorESM1-M_rcp85_r1i1p1.nc'], ['dataset/rcp85/BNU-ESM/daily_projections-cmip5-daily-single-levels_tasmin-tasmax_BNU-ESM_rcp85_r1i1p1.nc'], ['dataset/rcp85/MPI-ESM-MR/daily_projections-cmip5-daily-single-levels_tasmin-tasmax_MPI-ESM-MR_rcp85_r1i1p1.nc'], ['dataset/rcp85/MPI-ESM-LR/daily_projections-cmip5-daily-single-levels_tasmin-tasmax_MPI-ESM-L

,Array,Chunk
Bytes,175.67 MB,17.57 MB
Shape,"(5, 2, 1, 34310, 8, 8)","(1, 1, 1, 34310, 8, 8)"
Count,120 Tasks,10 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,175.67 MB,17.57 MB
Shape,"(5, 2, 1, 34310, 8, 8)","(1, 1, 1, 34310, 8, 8)"
Count,120 Tasks,10 Chunks
Type,float64,numpy.ndarray


### Create ZARR

In [0]:
import zarr
# Create zarr store structure
#source = zarr.ZipStore(, mode='w')
zarr_dir = "spain.zarr"
root = zarr.open(zarr_dir, mode='w')
#ds4 = ds.chunk(100)
ds.to_zarr(f"{zarr_dir}/projections-cmip5-daily-single-levels", "w")
zarr.consolidate_metadata(zarr_dir)
print(root.tree())

/
 └── projections-cmip5-daily-single-levels
     ├── experiment (2,) object
     ├── height (5,) float64
     ├── lat (8,) float64
     ├── lon (8,) float64
     ├── model (5,) object
     ├── realization (1,) int64
     ├── tasmax (5, 2, 1, 34333, 8, 8) float64
     ├── tasmin (5, 2, 1, 34333, 8, 8) float64
     └── time (34333,) int64


In [0]:
import xarray as xr
ds = xr.open_zarr(zarr_dir, group="projections-cmip5-daily-single-levels", consolidated=True)
ds

<xarray.Dataset>
Dimensions:      (experiment: 2, lat: 8, lon: 8, model: 5, realization: 1, time: 34333)
Coordinates:
  * experiment   (experiment) object 'rcp45' 'rcp85'
    height       (model) float64 dask.array<chunksize=(5,), meta=np.ndarray>
  * lat          (lat) float64 35.0 36.25 37.5 38.75 40.0 41.25 42.5 43.75
  * lon          (lon) float64 -9.375 -7.5 -5.625 -3.75 -1.875 0.0 1.875 3.75
  * model        (model) object 'NorESM1-M' 'BNU-ESM' ... 'ACCESS1-0'
  * realization  (realization) int64 1
  * time         (time) object 2006-01-01 00:00:00 ... 2100-01-23 00:00:00
Data variables:
    tasmax       (model, experiment, realization, time, lat, lon) float64 dask.array<chunksize=(1, 1, 1, 34333, 8, 8), meta=np.ndarray>
    tasmin       (model, experiment, realization, time, lat, lon) float64 dask.array<chunksize=(1, 1, 1, 34333, 8, 8), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    comment:      maximum near-surface (usually, 2 meter) air temperature (ad...
    source:       CMIP5, CDS

In [0]:
import xclim as xc
out = xc.indices.heat_wave_frequency(tasmax=ds.tasmax, tasmin=ds.tasmin, thresh_tasmax='25 C', thresh_tasmin='20 C', window=2, freq='MS')
# Prefer to use atmos as it does checks and uses units, but it complains about daily data 
#out = xc.atmos.heat_wave_frequency(tasmax=ds.tasmax, tasmin=ds.tasmin, thresh_tasmax='25 C', thresh_tasmin='20 C', window=2, freq='MS')
print(out)
# Plot
out2d = out.isel(time=7)
out2d.plot()

In [0]:
# Clean up
import os
import glob
dummy = [os.remove(f) for f in glob.glob('*.nc')]

## ERA5-Land-hourly-data-from-1981-to-present

In [0]:
%%writefile download.py
import cdstoolbox as ct

# Out directory
ds_dir = "dataset"

# Define region of interest
bbox = [35, -10, 44, 5]
grid = [0.1, 0.1]

# Define datasets to download
var_list = ["2m_temperature"]
times = [f"0{t}:00" if t < 10 else f"{t}:00"  for t in range(0, 23)]
days = [f"0{t}" if t < 10 else f"{t}"  for t in range(1, 31)]
months = [f"0{t}" if t < 10 else f"{t}"  for t in range(1, 12)]
years = [
             [str(t) for t in range(1980, 1989)],
             [str(t) for t in range(1990, 1999)],
             [str(t) for t in range(2000, 2009)],
             [str(t) for t in range(2010, 2019)] 
               ]

# Function to create properties dict for download
def create_prop_dict(variable, days, months, years, bbox, grid):
  return {
      "variable" : variable,
      "time" : times,
      "day" : days,
      "month" : months,
      "year" : years,
      'area': bbox,
      'grid': grid
      }

# Make list of properties dicts, and out names
pdict_list = []
nm_list = []
for variable in var_list:
  for year in years:
      pdict_list.append(create_prop_dict(variable, days, months, year, bbox, grid))
      # make file names
      if len(years) >1:
        ti = f"{year[0]}-{year[-1]}"
      else:
        ti = year[0] 
      v = variable.replace("_", "-")
      nm_list.append(f"bbox-{bbox[0]}-{bbox[1]}-{bbox[2]}-{bbox[3]}_{v}_{ti}.nc")

@ct.output.download()
def application():
    
    #ref_grid = ct.catalogue.retrieve(
    #    "reanalysis-era5-land",\
    #     {"variable" : "2m_temperature","time" : '12:00',"day" : '15',"month" : '08', "year" : '2000', 'area': bbox, 'grid': grid})

    def get_data(pdict, nm):
      ds = ct.catalogue.retrieve('reanalysis-era5-land', pdict)
      data_daily_max = ct.cdm.rename(ct.climate.daily_max(data),"tasmax")
      data_daily_min = ct.cdm.rename(ct.climate.daily_min(data),"tasmin")
      return [data_daily_max, data_daily_min]
    
    return [get_data(pdict, nm) for pdict, nm in zip(pdict_list, nm_list)]

Overwriting download.py


In [0]:
%%time
!python download.py

2020-03-03 16:21:44,912 INFO Welcome to the CDS
2020-03-03 16:21:44,913 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/tasks/services/tool/toolbox/orchestrator/run_workflow/clientid-2591abea5cac488495e12a53dda19eb4
2020-03-03 16:21:45,729 INFO Request is queued
2020-03-03 16:21:46,823 INFO Request is running


In [0]:
import xarray as xr
ds = xr.open_dataset()

### Rename netcdfs

In [0]:
import xarray as xr
import glob
import os
import pprint
xr.set_options(display_style='html')

fl = glob.glob("*.nc")
print(fl)
for f in fl:
  ds = xr.open_dataset(f)
  ds = xr.decode_cf(ds)
  attrs = ds.attrs
  pprint.pprint(attrs)
  if attrs['comment'].split(" ")[0] == 'maximum':
    v = 'tasmax'
  if attrs['comment'].split(" ")[0] == 'minimum':
    v = 'tasmin'
  dvattr = ds[v].attrs
  #pprint.pprint(dvattr)
  try:
    m = dvattr['model_id']
    e = dvattr['experiment_id']
    r = dvattr['parent_experiment_rip']
    nm = f"daily_projections-cmip5-daily-single-levels_{v}_{m}_{e}_{r}.nc"
  except:
    txt = dvattr['associated_files'].split('areacella:')[1]
    m = txt.split("_")[2]
    e = txt.split("_")[3]
    r = txt.split("_")[4]
    nm = f"daily_projections-cmip5-daily-single-levels_{v}_{m}_{e}_{r}"

  print(nm)
  ds.to_netcdf(nm)

['cdm.update_attributes-1583224022.0802572-17537-34-15ddf416-f6ad-4e4b-8aef-d69016eb0dcdupdated_attrs.nc', 'cdm.update_attributes-1583224309.65743-2661-30-180d1080-49aa-4c5b-b630-0c54353a7b72updated_attrs.nc', 'cdm.update_attributes-1583105491.5020332-14551-28-075dce16-6f10-4659-8f31-31540de29181updated_attrs.nc', 'cdm.update_attributes-1583224117.041823-11899-33-a1d207a6-cc60-4f06-9297-7ced56aee330updated_attrs.nc', 'cdm.update_attributes-1583224258.8644996-13789-38-ad5e2c57-7201-4277-ad46-482dcfe784fcupdated_attrs.nc', 'cdm.update_attributes-1583224316.5325725-26301-35-a450a1fc-0145-47a7-9f9a-b3714ce7b7b3updated_attrs.nc', 'cdm.update_attributes-1583224289.3021362-14066-38-2546732a-8551-448d-8fef-3bc414c497d5updated_attrs.nc', 'cdm.update_attributes-1583224124.7870688-16505-30-81064b06-ce98-4c7f-b8a6-da4b85f10b4dupdated_attrs.nc', 'cdm.update_attributes-1583224018.170093-32545-21-487bb3a9-27b3-4756-996a-8375a2b8b103updated_attrs.nc', 'cdm.update_attributes-1583224255.5815945-15857-38

In [0]:
# Copy new files to GS
copy_gcs([ds_dir], [gs_prefix], "-n")

Processing: gsutil -m cp -r -n dataset gs://copernicus-climate
Task created
Finished copy


## Thermal-comfort-indices-derived-from-ERA5-reanalysis

In [0]:
# Get data from CDS (this could take along time to order, but once ready quick to download)
# Get Thermal-comfort-indices-derived-from-ERA5-reanalysis
%%time
grid= [0.25, 0.25]
bbox= [44.0, -10.0, 35.0, 5.0]
ds_name = "Thermal-comfort-indices-derived-from-ERA5-reanalysis"
for y in range(1980, 1981):
  for m in range(1, 12):
    for d in range(1, 31):
      #try:
        application(ds = ds_name,
        bbox = bbox, # North, West, South, East. Default: global
        grid = grid, # Latitude/longitude grid: east-west (longitude) and north-south resolution (latitude). Default: 0.25 x 0.25
        start_year = y,
        end_year = y+1,
        start_month=m,
        end_month=m+1,
        start_day=d,
        end_day=d+1)
      #except:
        #pass     


Fetching derived-utci-historical_Mean radiant temperature_1980--1981.zip
With properties {'format': 'zip', 'variable': 'Mean radiant temperature', 'product_type': 'Consolidated dataset', 'time': ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00'], 'day': ['01'], 'month': ['01'], 'year': ['1980'], 'area': [44.0, -10.0, 35.0, 5.0], 'grid': [0.25, 0.25]}


2020-02-25 15:27:44,259 INFO Welcome to the CDS
2020-02-25 15:27:44,259 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/derived-utci-historical
2020-02-25 15:27:45,432 INFO Request is queued
2020-02-25 15:27:46,692 INFO Request is completed
2020-02-25 15:27:46,693 INFO Downloading http://136.156.132.105/cache-compute-0000/cache/downloads/cemsdata.copernicus-climate.eu/UTCI-HEALTH-HISTORICAL/reanalysis/1.0/1980/ECMWF_mrt_19800101_v1.0_con.nc to dataset/derived-utci-historical_Mean radiant temperature_1980--1981.zip (48.8M)
2020-02-25 15:27:56,584 INFO Download rate 4.9M/s


Fetching derived-utci-historical_Mean radiant temperature_1980--1981.zip
With properties {'format': 'zip', 'variable': 'Mean radiant temperature', 'product_type': 'Consolidated dataset', 'time': ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00'], 'day': ['02'], 'month': ['01'], 'year': ['1980'], 'area': [44.0, -10.0, 35.0, 5.0], 'grid': [0.25, 0.25]}


2020-02-25 15:27:58,201 INFO Welcome to the CDS
2020-02-25 15:27:58,204 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/derived-utci-historical
2020-02-25 15:27:59,285 INFO Request is queued
2020-02-25 15:28:00,552 INFO Request is completed
2020-02-25 15:28:00,553 INFO Downloading http://136.156.133.46/cache-compute-0015/cache/downloads/cemsdata.copernicus-climate.eu/UTCI-HEALTH-HISTORICAL/reanalysis/1.0/1980/ECMWF_mrt_19800102_v1.0_con.nc to dataset/derived-utci-historical_Mean radiant temperature_1980--1981.zip (48.7M)
2020-02-25 15:28:11,088 INFO Download rate 4.6M/s


Fetching derived-utci-historical_Mean radiant temperature_1980--1981.zip
With properties {'format': 'zip', 'variable': 'Mean radiant temperature', 'product_type': 'Consolidated dataset', 'time': ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00'], 'day': ['03'], 'month': ['01'], 'year': ['1980'], 'area': [44.0, -10.0, 35.0, 5.0], 'grid': [0.25, 0.25]}


...

Fetching derived-utci-historical_Mean radiant temperature_1980--1981.zip
With properties {'format': 'zip', 'variable': 'Mean radiant temperature', 'product_type': 'Consolidated dataset', 'time': ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00'], 'day': ['26'], 'month': ['02'], 'year': ['1980'], 'area': [44.0, -10.0, 35.0, 5.0], 'grid': [0.25, 0.25]}


2020-02-25 15:42:23,287 INFO Welcome to the CDS
2020-02-25 15:42:23,288 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/derived-utci-historical
2020-02-25 15:42:24,600 INFO Request is queued
2020-02-25 15:42:25,855 INFO Request is completed
2020-02-25 15:42:25,856 INFO Downloading http://136.156.133.42/cache-compute-0014/cache/downloads/cemsdata.copernicus-climate.eu/UTCI-HEALTH-HISTORICAL/reanalysis/1.0/1980/ECMWF_mrt_19800226_v1.0_con.nc to dataset/derived-utci-historical_Mean radiant temperature_1980--1981.zip (49.1M)
2020-02-25 15:42:36,138 INFO Download rate 4.8M/s


Fetching derived-utci-historical_Mean radiant temperature_1980--1981.zip
With properties {'format': 'zip', 'variable': 'Mean radiant temperature', 'product_type': 'Consolidated dataset', 'time': ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00'], 'day': ['27'], 'month': ['02'], 'year': ['1980'], 'area': [44.0, -10.0, 35.0, 5.0], 'grid': [0.25, 0.25]}


2020-02-25 15:42:37,417 INFO Welcome to the CDS
2020-02-25 15:42:37,418 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/derived-utci-historical
2020-02-25 15:42:38,703 INFO Request is queued
2020-02-25 15:42:39,960 INFO Request is completed
2020-02-25 15:42:39,961 INFO Downloading http://136.156.133.42/cache-compute-0014/cache/downloads/cemsdata.copernicus-climate.eu/UTCI-HEALTH-HISTORICAL/reanalysis/1.0/1980/ECMWF_mrt_19800227_v1.0_con.nc to dataset/derived-utci-historical_Mean radiant temperature_1980--1981.zip (49M)


KeyboardInterrupt: ignored

In [0]:
# Copy new files to GS
copy_gcs([ds_dir], [gs_prefix], "-n")

In [0]:
import cdsapi
#comment by Jorge
c = cdsapi.Client()

"""c.retrieve(
    'seasonal-postprocessed-single-levels',
    {
        'originating_centre':'ecmwf',
        'system':'5',
        'variable':'maximum_2m_temperature_in_the_last_24_hours_anomaly',
        'product_type':'ensemble_mean',
        'year':'2019',
        'month':'06',
        'leadtime_month':[
            '1','2','3',
            '4','5','6'
        ],
        'format':'grib'
    },
    'sea_an_ecmwf_5_t2max_1906_6.grib')"""

"""c.retrieve(
    'seasonal-postprocessed-single-levels',
    {
        'originating_centre':'ukmo',
        'system':'14',
        'variable':'maximum_2m_temperature_in_the_last_24_hours_anomaly',
        'product_type':'ensemble_mean',
        'year':'2019',
        'month':'06',
        'leadtime_month':[
            '1','2','3',
            '4','5','6'
        ],
        'format':'grib'
    },
    'sea_an_ukmo_14_t2max_1906_6.grib')"""

"""c.retrieve(
    'seasonal-postprocessed-single-levels',
    {
        'originating_centre':'meteo_france',
        'system':'6',
        'variable':'maximum_2m_temperature_in_the_last_24_hours_anomaly',
        'product_type':'ensemble_mean',
        'year':'2019',
        'month':'06',
        'leadtime_month':[
            '1','2','3',
            '4','5','6'
        ],
        'format':'grib'
    },
    'sea_an_meteofrance_6_t2max_1906_6.grib')"""


c.retrieve(
    'seasonal-postprocessed-single-levels',
    {
        'originating_centre':'dwd',
        'system':'2',
        'variable':'maximum_2m_temperature_in_the_last_24_hours_anomaly',
        'product_type':'ensemble_mean',
        'year':'2019',
        'month':'06',
        'leadtime_month':[
            '1','2','3',
            '4','5','6'
        ],
        'format':'grib'
    },
    'sea_an_dwd_2_t2max_1906_6.grib')

c.retrieve(
    'seasonal-postprocessed-single-levels',
    {
        'originating_centre':'cmcc',
        'system':'3',
        'variable':'maximum_2m_temperature_in_the_last_24_hours_anomaly',
        'product_type':'ensemble_mean',
        'year':'2019',
        'month':'06',
        'leadtime_month':[
            '1','2','3',
            '4','5','6'
        ],
        'format':'grib'
    },
    'sea_an_cmcc_3_t2max_1906_6.grib')
####################################################
c.retrieve(
    'seasonal-postprocessed-single-levels',
    {
        'originating_centre':'ecmwf',
        'system':'5',
        'variable':'minimum_2m_temperature_in_the_last_24_hours_anomaly',
        'product_type':'ensemble_mean',
        'year':'2019',
        'month':'06',
        'leadtime_month':[
            '1','2','3',
            '4','5','6'
        ],
        'format':'grib'
    },
    'sea_an_ecmwf_5_t2min_1906_6.grib')

c.retrieve(
    'seasonal-postprocessed-single-levels',
    {
        'originating_centre':'ukmo',
        'system':'14',
        'variable':'minimum_2m_temperature_in_the_last_24_hours_anomaly',
        'product_type':'ensemble_mean',
        'year':'2019',
        'month':'06',
        'leadtime_month':[
            '1','2','3',
            '4','5','6'
        ],
        'format':'grib'
    },
    'sea_an_ukmo_14_t2min_1906_6.grib')

c.retrieve(
    'seasonal-postprocessed-single-levels',
    {
        'originating_centre':'meteo_france',
        'system':'6',
        'variable':'minimum_2m_temperature_in_the_last_24_hours_anomaly',
        'product_type':'ensemble_mean',
        'year':'2019',
        'month':'06',
        'leadtime_month':[
            '1','2','3',
            '4','5','6'
        ],
        'format':'grib'
    },
    'sea_an_meteofrance_6_t2min_1906_6.grib')


c.retrieve(
    'seasonal-postprocessed-single-levels',
    {
        'originating_centre':'dwd',
        'system':'2',
        'variable':'minimum_2m_temperature_in_the_last_24_hours_anomaly',
        'product_type':'ensemble_mean',
        'year':'2019',
        'month':'06',
        'leadtime_month':[
            '1','2','3',
            '4','5','6'
        ],
        'format':'grib'
    },
    'sea_an_dwd_2_t2min_1906_6.grib')

c.retrieve(
    'seasonal-postprocessed-single-levels',
    {
        'originating_centre':'cmcc',
        'system':'3',
        'variable':'minimum_2m_temperature_in_the_last_24_hours_anomaly',
        'product_type':'ensemble_mean',
        'year':'2019',
        'month':'06',
        'leadtime_month':[
            '1','2','3',
            '4','5','6'
        ],
        'format':'grib'
    },
    'sea_an_cmcc_3_t2min_1906_6.grib')

In [0]:
!unzip /content/dataset/derived-utci-historical_mrt-uti_1980--1981.zip

Archive:  /content/dataset/derived-utci-historical_mrt-uti_1980--1981.zip
 bunzipping: ECMWF_utci_19800101_v1.0_con.nc  
 bunzipping: ECMWF_mrt_19800101_v1.0_con.nc  


In [0]:
import xarray as xr


In [0]:
!pip install netcdf4

     |████████████████████████████████| 4.1MB 2.7MB/s 
     |████████████████████████████████| 317kB 37.7MB/s 


In [0]:
ds = xr.open_dataset("/content/ECMWF_mrt_19800101_v1.0_con.nc")
ds

<xarray.Dataset>
Dimensions:  (lat: 601, lon: 1440, time: 24)
Coordinates:
  * time     (time) datetime64[ns] 1980-01-01 ... 1980-01-01T23:00:00
  * lon      (lon) float64 -180.0 -179.8 -179.5 -179.2 ... 179.2 179.5 179.8
  * lat      (lat) float64 90.0 89.75 89.5 89.25 ... -59.25 -59.5 -59.75 -60.0
Data variables:
    mrt      (time, lat, lon) float32 ...
Attributes:
    CDI:                       Climate Data Interface version 1.9.5 (http://m...
    Conventions:               CF-1.6
    history:                   Tue Dec 17 13:48:27 2019: cdo -P 8 -z zip_1 -R...
    institution:               European Centre for Medium-Range Weather Forec...
    CDO:                       Climate Data Operators version 1.9.5 (http://m...
    cdo_openmp_thread_number:  8

In [0]:
import pandas as pd
cat_df = pd.read_csv('https://storage.googleapis.com/pangeo-cmip6/pangeo-cmip6-zarr-consolidated-stores.csv')
cat_df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
0,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrbc,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN
1,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrdust,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN
2,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmroa,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN
3,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrso4,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN
4,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,Amon,rsut,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN


In [0]:
!pip install intake

     |████████████████████████████████| 3.1MB 2.6MB/s 
  Created wheel for intake: filename=intake-0.5.4-cp36-none-any.whl size=1457185 sha256=4a2987be50a4f50740a89040c965332087daad1e6e642fe6fbd3243cb181dcee
  Stored in directory: /root/.cache/pip/wheels/ac/90/37/b2aac45ddc4463e6809a10e4c8f3d579a9729ee3d4cb924203
  Created wheel for locket: filename=locket-0.2.0-cp36-none-any.whl size=4040 sha256=f4626813e4201c4526201b16df857e49afd6f2dd6592191033d3264b736c666e
  Stored in directory: /root/.cache/pip/wheels/26/1e/e8/4fa236ec931b1a0cdd61578e20d4934d7bf188858723b84698
Successfully built intake locket
